In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import fastf1
import plotly.express as px
import fastf1.plotting
import datetime

fastf1.Cache.enable_cache('../f1_cache')  

In [11]:
fastf1_session = fastf1.get_session(2021, 'Monza', 'Race')
fastf1_session.load(telemetry=True, laps=True, weather=False)

core           INFO 	Loading data for Italian Grand Prix - Race [v2.2.3]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
core           INFO 	Finished loading data for 20 drivers: ['3', '4', '77', '16', '11', '55', '18', '14', '63', '31', '6', '5', '99', '88', '47', '9', '44', '33', '10', '22']


In [4]:
# Getting df for first plot
lap_time_number = fastf1_session.laps.iloc[:, 1:4].copy()
lap_time_number_piv = lap_time_number.pivot(index="LapNumber", columns='DriverNumber', values='LapTime').rename_axis(None, axis=1).reset_index()
lap_time_number_cov = lap_time_number_piv.copy()

In [106]:
for col in lap_time_number_cov.columns[1:]:
    lap_time_number_cov[col] = pd.to_numeric(lap_time_number_cov[col])
    # lap_time_number_cov[col] = lap_time_number_cov[col]/1000000

## Speed vs Distance

In [63]:
selected_drivers = ['HAM', 'ALO', 'LAT']
final_dict = {}
for driver in selected_drivers:
    driv_tel = {}
    driv_lap = fastf1_session.laps.pick_driver(driver).pick_fastest()
    color = fastf1.plotting.team_color(driv_lap['Team'])
    driv_tel['Data'] = driv_lap.get_car_data().add_distance()[['Speed', 'Distance']].rename(columns={"Speed": driver}).to_dict('records')
    driv_tel['Color'] = color
    final_dict[driver] = driv_tel
    # driv_tel[driver] = 

In [73]:
driv_lap = fastf1_session.laps.pick_driver("HAM").pick_fastest()
driv_lap.get_car_data()


,Date,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Time,SessionTime
0,2021-09-12 13:06:29.974,11315,313,8,100,False,1,car,0 days 00:00:00.231000,0 days 01:05:28.365000
1,2021-09-12 13:06:30.214,11393,315,8,100,False,1,car,0 days 00:00:00.471000,0 days 01:05:28.605000
2,2021-09-12 13:06:30.454,11481,317,8,100,False,1,car,0 days 00:00:00.711000,0 days 01:05:28.845000
3,2021-09-12 13:06:30.694,11509,318,8,100,False,1,car,0 days 00:00:00.951000,0 days 01:05:29.085000
4,2021-09-12 13:06:30.934,11573,320,8,100,False,1,car,0 days 00:00:01.191000,0 days 01:05:29.325000
...,...,...,...,...,...,...,...,...,...,...
351,2021-09-12 13:07:54.415,10978,302,8,100,False,0,car,0 days 00:01:24.672000,0 days 01:06:52.806000
352,2021-09-12 13:07:54.655,11002,304,8,100,False,0,car,0 days 00:01:24.912000,0 days 01:06:53.046000
353,2021-09-12 13:07:54.895,11134,306,8,100,False,0,car,0 days 00:01:25.152000,0 days 01:06:53.286000
354,2021-09-12 13:07:55.135,11157,309,8,100,False,0,car,0 days 00:01:25.392000,0 days 01:06:53.526000
